# Data prepration

In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats

2022-07-20 16:08:08.631801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 16:08:08.631832: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test


def discrete_angles(n, m, s, rand = 1234):
    angles = np.random.uniform(0, s * np.pi, m)
    xk = np.arange(m)
    pk = (1 / m) * np.ones(int(m))
    discrete_distr = stats.rv_discrete(name='discrete_distr', values=(xk, pk), seed=rand)
    thetas = angles[discrete_distr.rvs(size=n)]
    return thetas

In [3]:
n = 50000


# m_th = 15
# m_ph = 15
# thetas = discrete_angles(n, m_th, 2)
# phis = discrete_angles(n, m_ph, 1)
# name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))

theta = 3
phi = 3
thetas = theta * np.ones(n)
phis = phi * np.ones(n)
name = ('discrete_model_th%.0f_ph%.0f' %(theta, phi))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[ 1.25310791  0.11627582  3.          3.        ]
 [ 0.64450199 -0.89520966  3.          3.        ]
 [-0.22159352 -0.78304374  3.          3.        ]
 ...
 [-1.16694033 -1.00272209  3.          3.        ]
 [ 0.77792134 -0.87131836  3.          3.        ]
 [-0.75541539 -1.07913565  3.          3.        ]]


# Training Interpolated Model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 1000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=10
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-07-20 16:08:13.098529: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-20 16:08:13.098559: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-20 16:08:13.098577: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-07-20 16:08:13.098822: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-20 16:08:13.110388: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initiali

Epoch 1/10
22/80 [=======>......................] - ETA: 0s - loss: 0.6065 - accuracy: 0.6682

2022-07-20 16:08:13.827621: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-07-20 16:08:13.827649: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-07-20 16:08:13.839941: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-07-20 16:08:13.841839: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-07-20 16:08:13.854763: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220720-160813/train/plugins/profile/2022_07_20_16_08_13

2022-07-20 16:08:13.860255: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220720-160813/train/plugins/profile/2022_07_20_16_08_13/jupyter-matan.trace.json.gz
2022-07-20 16:08:13.864485: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220720-160

80/80 [==============================] - 1s 5ms/step - loss: 0.5780 - accuracy: 0.6953
Epoch 2/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5651 - accuracy: 0.7062
Epoch 3/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5644 - accuracy: 0.7063
Epoch 4/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5644 - accuracy: 0.7069
Epoch 5/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5636 - accuracy: 0.7071
Epoch 6/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5633 - accuracy: 0.7069
Epoch 7/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5634 - accuracy: 0.7065
Epoch 8/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5634 - accuracy: 0.7071
Epoch 9/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5636 - accuracy: 0.7068
Epoch 10/10
80/80 [==============================] - 0s 4ms/step - loss: 0.5628 - accuracy: 0.7072


2022-07-20 16:08:17.293576: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 3dmodels/discrete_model_th3_ph3/assets


In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)